In [1]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import matplotlib.pyplot as plt
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [3]:
key = {'subject_uuid': UUID('034fd43f-ee35-450b-99ef-25475ce25fdc'), 'last_session_date': datetime.date(2019, 2, 25)}
# get trial counts and session length to date
sessions = (behavior.BehavioralSummaryByDate.PsychResults & 'prob_left=0.5' &
            key).proj('session_date', 'signed_contrasts', 'prob_choose_right').fetch(as_dict=True)
# reshape to a heatmap format
contrast_list = []
for session in sessions:
    for icontrast, contrast in enumerate(session['signed_contrasts']):
        contrast_list.append({'session_date': session['session_date'], 
                             'signed_contrast': round(contrast, 2)*100, 
                             'prob_choose_right': session['prob_choose_right'][icontrast]})
contrast_df = pd.DataFrame(contrast_list)
contrast_map = contrast_df.pivot(
    'signed_contrast', 'session_date', 'prob_choose_right').sort_values(
        by='signed_contrast', ascending=False)

contrast_map = contrast_map.where((pd.notnull(contrast_map)), None)

In [4]:
data = dict(
    x=[t.strftime('%Y-%m-%d') 
       for t in contrast_map.columns.tolist()],
    y=contrast_map.index.tolist(),
    z=contrast_map.values.tolist(),
    zmax=1,
    zmin=0,
    type='heatmap',
    colorbar=dict(
        thickness=10,
        title='prob choosing left',
        titleside = 'right',
    )
    
)

layout = go.Layout(
    xaxis=dict(title='Date'),
    yaxis=dict(
        title='Contrast (%)',
        range=[-100, 100]
    ),
    width=500,
    height=400,
    title='Contrast heatmap',
    showlegend=False
)

fig = go.Figure(data=[data], layout=layout)
plotly.offline.iplot(fig)

In [150]:
f = open("contrast_heatmap.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()